In [ ]:
# default_exp roster

In [ ]:
%load_ext autoreload
%autoreload 2

# Roster

> Module to parse and model a roster with different shifts.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export
from dataclasses import dataclass
from datetime import datetime, timedelta
from ics import Calendar, Event
import re
from typing import Any, Optional


@dataclass
class ShiftProperties:
    name: str
    starting_hour: timedelta
    duration: timedelta


@dataclass
class Shift:
    properties: ShiftProperties
    date: datetime

    def __post_init__(self):
        self.beginning: datetime = self.date + self.properties.starting_hour


In [ ]:
sp = ShiftProperties(
    name="Früh0", starting_hour=timedelta(hours=7), duration=timedelta(hours=8)
)
Shift(properties=sp, date=datetime(2022, 1, 1, 7, 0))


Shift(properties=ShiftProperties(name='Früh0', starting_hour=datetime.timedelta(seconds=25200), duration=datetime.timedelta(seconds=28800)), date=datetime.datetime(2022, 1, 1, 7, 0))

In [ ]:
# export
@dataclass
class Roster:
    shifts: list[Shift]
    name: str = "Jane Doe"
    _year: int = 2022
    _month: int = 1  # TODO: Read from Excel
    _dayp = re.compile(r"MO|DI|MI|DO|FR|SA|SO")
    _datep = re.compile(r"\d{2}")

    @classmethod
    def from_dict(
        cls, input: dict[str, str], mapper: Optional[dict] = None
    ) -> "Roster":
        shifts = []
        for date_str, abbr in input.items():
            props = mapper[abbr]
            if props is None:
                continue
            date = datetime(
                year=cls._year,
                month=cls._month,
                day=int(cls._datep.search(date_str).group()),
            )
            shift = Shift(props, date=date)
            shifts.append(shift)
        return cls(shifts=shifts)

    def to_ics(self):
        c = Calendar()
        for shift in self.shifts:
            e = Event()
            e.name = shift.properties.name
            e.begin = shift.beginning
            e.duration = shift.properties.duration
            c.events.add(e)
        return c
